#Removendo a duplicidade de dados


## Tabela UF

**Descrição**: Remove os espaços em branco e compara os nomes dos estados em letras minúsculas.

**Saída**: Retorna os dados duplicados, indicando quantas vezes cada um se repetiu, além do nome e ID da primeira ocorrência na tabela.



In [1]:
%%bigquery
WITH cte_uf_duplicados AS (
  SELECT
    id_uf,
    uf,
    LOWER(REPLACE(uf, ' ', '')) AS uf_formatado,
    ROW_NUMBER() OVER(
      PARTITION BY LOWER(REPLACE(uf, ' ', ''))
      ORDER BY 1
    ) AS qtde
  FROM `marketplace-challenge-456003.staging.uf`
)

SELECT
    d1.id_uf,
    d1.uf_formatado,
    d1.qtde,

    d2.id_uf,
    d2.uf_formatado,
    d2.qtde

FROM cte_uf_duplicados d1
INNER JOIN cte_uf_duplicados d2 ON (d1.uf_formatado = d2.uf_formatado)
WHERE d1.qtde > 1
AND d2.qtde = 1
ORDER BY 1


Query is running:   0%|          |

Downloading: |          |

,id_uf,uf_formatado,qtde,id_uf_1,uf_formatado_1,qtde_1


**Descrição**: Para remover os estados duplicados é necessário veririficar se o id referente a esses dados está vinculado a algum registro da tabela municipio

In [2]:
%%bigquery
UPDATE `marketplace-challenge-456003.staging.municipio` c
SET c.id_uf = d2.id_uf
FROM (
  SELECT
    id_uf,
    uf,
    LOWER(REPLACE(uf, ' ', '')) AS uf_formatado,
    ROW_NUMBER() OVER(
      PARTITION BY LOWER(REPLACE(uf, ' ', ''))
      ORDER BY id_uf
    ) AS qtde
  FROM `marketplace-challenge-456003.staging.uf`
) d1
INNER JOIN (
  SELECT
    id_uf,
    uf,
    LOWER(REPLACE(uf, ' ', '')) AS uf_formatado,
    ROW_NUMBER() OVER(
      PARTITION BY LOWER(REPLACE(uf, ' ', ''))
      ORDER BY id_uf
    ) AS qtde
  FROM `marketplace-challenge-456003.staging.uf`
) d2
ON d1.uf_formatado = d2.uf_formatado
WHERE d1.qtde > 1
AND d2.qtde = 1
AND c.id_uf = d1.id_uf;

-- 47 linhas alteradas


Query is running:   0%|          |

""


Deletando os UF duplicadas

In [3]:
%%bigquery

DELETE
FROM `marketplace-challenge-456003.staging.uf`
WHERE id_uf IN (
    SELECT d1.id_uf
    FROM
    (
        SELECT
            id_uf,
            uf,
            LOWER(REPLACE(uf, ' ', '')) AS uf_normalizado,
            ROW_NUMBER() OVER(
            PARTITION BY LOWER(REPLACE(uf, ' ', ''))
            ORDER BY 1
            ) AS qtde
        FROM `marketplace-challenge-456003.staging.uf`
    ) d1
    INNER JOIN
    (
        SELECT
            id_uf,
            uf,
            LOWER(REPLACE(uf, ' ', '')) AS uf_normalizado,
            ROW_NUMBER() OVER(
            PARTITION BY LOWER(REPLACE(uf, ' ', ''))
            ORDER BY 1
            ) AS qtde
        FROM `marketplace-challenge-456003.staging.uf`

    ) d2
    ON (d1.uf_normalizado = d2.uf_normalizado)
    WHERE d1.qtde > 1
    AND d2.qtde = 1
)
-- 21 linhas


Query is running:   0%|          |

""


Conferindo se realmente foram deletados

In [4]:
%%bigquery
WITH cte_uf_duplicados AS (
  SELECT
    id_uf,
    uf,
    LOWER(REPLACE(uf, ' ', '')) AS uf_formatado,
    ROW_NUMBER() OVER(
      PARTITION BY LOWER(REPLACE(uf, ' ', ''))
      ORDER BY 1
    ) AS qtde
  FROM `marketplace-challenge-456003.staging.uf`
)

SELECT
    d1.id_uf,
    d1.uf_formatado,
    d1.qtde,

    d2.id_uf,
    d2.uf_formatado,
    d2.qtde

FROM cte_uf_duplicados d1
INNER JOIN cte_uf_duplicados d2 ON (d1.uf_formatado = d2.uf_formatado)
WHERE d1.qtde > 1
AND d2.qtde = 1
ORDER BY 1


Query is running:   0%|          |

Downloading: |          |

,id_uf,uf_formatado,qtde,id_uf_1,uf_formatado_1,qtde_1


Conferindo os dados dentro na tabela UF

In [5]:
%%bigquery
SELECT *
FROM `marketplace-challenge-456003.staging.uf`
ORDER BY uf

Query is running:   0%|          |

Downloading:   0%|          |

,id_uf,uf
0,2,AC
1,36,AL
2,4,AM
3,5,AP
4,6,BA
5,37,CE
6,38,DF
7,10,ES
8,12,GO
9,39,MA


Colocando todos os estados em letra Maiuscula

In [6]:
%%bigquery
UPDATE `marketplace-challenge-456003.staging.uf`
SET uf = upper(uf)
where 1=1


Query is running:   0%|          |

""


Deletando o estado '31' de id = 1

In [7]:
%%bigquery
SELECT *
FROM `marketplace-challenge-456003.staging.municipio`
where id_uf = 1

Query is running:   0%|          |

Downloading: |          |

,id_municipio,municipio,id_uf


Verificando se o id=1 UF='31' tem alguma referência na tabela Municipio antes de ser deletado

In [8]:
%%bigquery
SELECT *
FROM `marketplace-challenge-456003.staging.municipio`
where municipio like '%etim'

Query is running:   0%|          |

Downloading:   0%|          |

,id_municipio,municipio,id_uf
0,1058,Betim,14


Atualizando o id_uf na tabela municipio para deletar a uf

In [9]:
%%bigquery
UPDATE `marketplace-challenge-456003.staging.municipio`
SET id_uf = 14
where id_uf = 1

Query is running:   0%|          |

""


Deletando o UF incorreto

In [10]:
%%bigquery
DELETE
FROM `marketplace-challenge-456003.staging.uf`
WHERE id_uf = 1

Query is running:   0%|          |

""


In [11]:
%%bigquery
SELECT *
FROM `marketplace-challenge-456003.staging.uf`
ORDER BY uf

Query is running:   0%|          |

Downloading:   0%|          |

,id_uf,uf
0,2,AC
1,36,AL
2,4,AM
3,5,AP
4,6,BA
5,37,CE
6,38,DF
7,10,ES
8,12,GO
9,39,MA


## Tabela Municipio

In [12]:
%%bigquery

WITH cte_municipios_duplicadas AS (
  SELECT
    id_municipio,
    municipio,
    id_uf,
    LOWER(REPLACE(municipio, ' ', '')) AS municipio_formatado,
    ROW_NUMBER() OVER (
      PARTITION BY LOWER(REPLACE(municipio, ' ', '')), id_uf
    ) AS qtde
  FROM `marketplace-challenge-456003.staging.municipio`
)

SELECT *
FROM cte_municipios_duplicadas cd
WHERE cd.qtde > 1
ORDER BY municipio_formatado
-- 692 linhas duplicadas

Query is running:   0%|          |

Downloading:   0%|          |

,id_municipio,municipio,id_uf,municipio_formatado,qtde
0,20,ABREU E LIMA,20,abreuelima,2
1,39,ADAMANTINA,33,adamantina,2
2,46,AFOGADOS DA INGAZEIRA,20,afogadosdaingazeira,2
3,7548,afogados da ingazeira,20,afogadosdaingazeira,3
4,525,Aguai,33,aguai,2
...,...,...,...,...,...
687,7516,Volta Redonda,24,voltaredonda,2
688,7481,VOTORANTIM,33,votorantim,2
689,7482,VOTUPORANGA,33,votuporanga,2
690,7543,Xaxim,31,xaxim,2


## NOVA CONSULTA DUPLICADOS MUNICIPIO

In [ ]:
%%bigquery
WITH cte_municipios_duplicados AS (
  SELECT
      id_municipio,
      municipio,
      id_uf,
      -- Formata o município removendo caracteres especiais E NÚMEROS (mas mantendo acentuações e letras)
      LOWER(REGEXP_REPLACE(municipio, '[^\\p{L}]', '')) AS municipio_formatado,
      ROW_NUMBER() OVER (
          PARTITION BY LOWER(REGEXP_REPLACE(municipio, '[^\\p{L}]', '')), id_uf
          ORDER BY id_municipio ASC -- Garante um resultado determinístico
      ) AS qtde
  FROM
      `marketplace-challenge-456003.staging.municipio`
)

SELECT *
FROM cte_municipios_duplicados cd1
INNER JOIN cte_municipios_duplicados cd2
      ON (
        cd1.municipio_formatado = cd2.municipio_formatado
        AND  cd1.id_uf = cd2.id_uf
      )
WHERE cd1.qtde > 1
AND cd2.qtde = 1
-- 714 linhas

In [ ]:
%%bigquery
WITH cte_municipios_duplicadas AS (
    SELECT
        id_municipio,
        municipio,
        id_uf,
        -- Formata o município removendo caracteres especiais E NÚMEROS (mas mantendo acentuações e letras)
        LOWER(REGEXP_REPLACE(municipio, '[^\\p{L}]', '')) AS municipio_formatado_limpo,
        ROW_NUMBER() OVER (
            PARTITION BY LOWER(REGEXP_REPLACE(municipio, '[^\\p{L}]', '')), id_uf
            ORDER BY id_municipio ASC -- Garante um resultado determinístico
        ) AS qtde_duplicidade
    FROM
        `marketplace-challenge-456003.staging.municipio`

)

SELECT *
FROM cte_municipios_duplicadas cd
WHERE cd.qtde_duplicidade > 1
ORDER BY municipio_formatado_limpo
-- 692 linhas duplicadas

In [ ]:
%%bigquery
SELECT
    id_municipio,
    municipio,
    id_uf,
    -- Formata o município removendo caracteres especiais e espaços, e convertendo para minúsculas
    LOWER(REGEXP_REPLACE(municipio, '[^a-zA-Z0-9]', '')) AS municipio_formatado_limpo,
    ROW_NUMBER() OVER (
        PARTITION BY LOWER(REGEXP_REPLACE(municipio, '[^a-zA-Z0-9]', '')), id_uf
        ORDER BY id_municipio ASC -- Adicione ORDER BY para resultados determinísticos
    ) AS qtde_duplicidade
FROM
    `marketplace-challenge-456003.staging.municipio`;




In [13]:
%%bigquery

WITH cte_municipios_duplicados AS (
  SELECT
    id_municipio,
    municipio,
    id_uf,
    LOWER(REPLACE(municipio, ' ', '')) AS municipio_formatado,
    ROW_NUMBER() OVER (
      PARTITION BY LOWER(REPLACE(municipio, ' ', '')), id_uf
    ) AS qtde
  FROM `marketplace-challenge-456003.staging.municipio`
)

SELECT *
FROM cte_municipios_duplicados cd1
INNER JOIN cte_municipios_duplicados cd2
      ON (cd1.municipio_formatado = cd2.municipio_formatado)
WHERE cd1.qtde > 1
AND cd2.qtde = 1
-- 714 linhas

Query is running:   0%|          |

Downloading:   0%|          |

,id_municipio,municipio,id_uf,municipio_formatado,qtde,id_municipio_1,municipio_1,id_uf_1,municipio_formatado_1,qtde_1
0,580,Assis,33,assis,2,483,ASSIS,33,assis,1
1,2457,FRANCO DA ROCHA,33,francodarocha,2,2495,Franco da Rocha,33,francodarocha,1
2,7587,franco da rocha,33,francodarocha,3,2495,Franco da Rocha,33,francodarocha,1
3,2569,GOIANIA,11,goiania,2,2729,Goiania,11,goiania,1
4,5662,RIO GRANDE DA SERRA,33,riograndedaserra,2,7634,rio grande da serra,49,riograndedaserra,1
...,...,...,...,...,...,...,...,...,...,...
709,3214,ITU,33,itu,2,3307,Itu,33,itu,1
710,5414,Pinhalzinho,31,pinhalzinho,2,5079,PINHALZINHO,31,pinhalzinho,1
711,7654,são borja,28,sãoborja,2,6855,São Borja,28,sãoborja,1
712,7543,Xaxim,31,xaxim,2,7665,xaxim,31,xaxim,1


Verificar as filiais vinculadas aos IDs de cidades duplicadas

Criação de tabela temporaria para testes com filial e municipio

In [14]:
%%bigquery
CREATE OR REPLACE table `marketplace-challenge-456003.staging.municipio_duplicado` AS
WITH cte_municipios_duplicados AS (
  SELECT
    id_municipio,
    municipio,
    id_uf,
    LOWER(REPLACE(municipio, ' ', '')) AS municipio_formatado,
    -- Adicione um ORDER BY aqui para garantir um resultado determinístico
    ROW_NUMBER() OVER (
      PARTITION BY LOWER(REPLACE(municipio, ' ', '')), id_uf
      ORDER BY id_municipio ASC -- Por exemplo, o menor id_municipio será o "original"
    ) AS qtde
  FROM `marketplace-challenge-456003.staging.municipio`
)
SELECT
  cd1.id_municipio AS id_municipio_dup,
  cd1.municipio AS municipio_dup,
  cd1.id_uf AS uf_dup,
  cd2.id_municipio AS id_municipio_inicial,
  cd2.municipio AS municipio_inicial,
  cd2.id_uf AS uf_inicial
FROM cte_municipios_duplicados cd1
INNER JOIN cte_municipios_duplicados cd2
  ON (cd1.municipio_formatado = cd2.municipio_formatado
      AND cd1.id_uf = cd2.id_uf
  )
WHERE cd1.qtde > 1
  AND cd2.qtde = 1
ORDER BY cd1.id_municipio;

Query is running:   0%|          |

""


In [15]:
%%bigquery
UPDATE `marketplace-challenge-456003.staging.filial` f
SET f.id_municipio = md.id_municipio_inicial
FROM `marketplace-challenge-456003.staging.municipio_duplicado` md
where f.id_municipio = md.id_municipio_dup
-- 3 linhas afetadas


SELECT
    f.id_filial, -- Assumindo que 'id_filial' é a chave primária da tabela 'filial'
    f.id_municipio AS id_municipio_antigo,
    md.id_municipio_inicial AS id_municipio_novo,
    --f.nome_filial, -- Inclua outras colunas da filial que sejam relevantes
    md.municipio_dup,
    md.municipio_inicial
FROM
    `marketplace-challenge-456003.staging.filial` f
JOIN
    `marketplace-challenge-456003.staging.municipio_duplicado` md
ON
    f.id_municipio = md.id_municipio_dup;

Executing query with job ID: 74f089c3-f0ee-45dd-8de1-673f98e72057
Query executing: 0.15s


ERROR:
 400 Syntax error: Expected end of input but got keyword SELECT at [8:1]; reason: invalidQuery, location: query, message: Syntax error: Expected end of input but got keyword SELECT at [8:1]

Location: US
Job ID: 74f089c3-f0ee-45dd-8de1-673f98e72057



FInalmente apaga os municipios duplicados

In [ ]:
%%bigquery
SELECT
    m.* -- Seleciona todas as colunas do município que seria deletado
FROM
    `marketplace-challenge-456003.staging.municipio` m
JOIN
    `marketplace-challenge-456003.staging.municipio_duplicado` md
ON
    m.id_municipio = md.id_municipio_dup;


DELETE `marketplace-challenge-456003.staging.municipio`
WHERE id_municipio IN (
    SELECT id_municipio_dup
    FROM `marketplace-challenge-456003.staging.municipio_duplicado`
);
-- 692 linhas afetadas


### Formata o nome de todos os municipios

In [ ]:
%%bigquery
UPDATE `marketplace-challenge-456003.staging.municipio`
SET municipio = INITCAP(municipio)
WHERE 1=1
-- 6978 linhas

### Apaga tabela temporária

In [ ]:
# drop table  `marketplace-challenge-456003.staging.municipio_duplicado`

## Tabela Filial

In [ ]:
%%bigquery

WITH cte_filiais_duplicadas AS (
  SELECT
    id_filial,
    ds_filial,
    id_cidade,
    LOWER(REPLACE(ds_filial, ' ', '')) AS ds_filial_formatado,
    ROW_NUMBER() OVER (
      PARTITION BY LOWER(REPLACE(ds_filial, ' ', '')), id_cidade
    ) AS qtde
  FROM `luizalabs-data-challenge.staging_corrigido.filial`
)

SELECT *
FROM cte_filiais_duplicadas fd1
WHERE fd1.qtde > 1
-- 722 linhas

## Tabela Parceiro

In [ ]:
%%bigquery

WITH cte_parceiros_duplicados AS (
  SELECT
    LOWER(REPLACE(nm_parceiro, ' ', '')) AS nm_parceiro_formatado,
    ROW_NUMBER() OVER (
      PARTITION BY LOWER(REPLACE(nm_parceiro, ' ', ''))
    ) AS qtde
  FROM `luizalabs-data-challenge.staging_corrigido.parceiro`
)

SELECT *
FROM cte_parceiros_duplicados pc
WHERE pc.qtde > 1

## Tabela Cliente

In [ ]:
%%bigquery
WITH cte_clientes_duplicados AS (
  SELECT
    LOWER(REPLACE(nm_cliente, ' ', '')) AS nm_cliente_formatado,
    ROW_NUMBER() OVER (
      PARTITION BY LOWER(REPLACE(nm_cliente, ' ', '')), flag_ouro
    ) AS qtde
  FROM `luizalabs-data-challenge.staging_corrigido.cliente`
)

SELECT *
FROM cte_clientes_duplicados cd
WHERE cd.qtde > 1


## Tabela Categoria

In [ ]:
%%bigquery
WITH cte_categorias_duplicados AS (
  SELECT
    LOWER(REPLACE(ds_categoria, ' ', '')) AS ds_categoria_formatada,
    ROW_NUMBER() OVER (
      PARTITION BY LOWER(REPLACE(ds_categoria, ' ', ''))
    ) AS qtde
  FROM `luizalabs-data-challenge.staging_corrigido.categoria`
)

SELECT *
FROM cte_categorias_duplicados cd
WHERE cd.qtde > 1



## Tabela Subcategoria

In [ ]:
%%bigquery
WITH cte_categorias_duplicados AS (
  SELECT
    LOWER(REPLACE(ds_categoria, ' ', '')) AS ds_categoria_formatada,
    ROW_NUMBER() OVER (
      PARTITION BY LOWER(REPLACE(ds_categoria, ' ', ''))
    ) AS qtde
  FROM `luizalabs-data-challenge.staging_corrigido.categoria`
)

SELECT *
FROM cte_categorias_duplicados cd
WHERE cd.qtde > 1


## Tabela Produto

In [ ]:
%%bigquery
WITH cte_produtos_duplicados AS (
  SELECT
    LOWER(REPLACE(ds_produto, ' ', '')) AS ds_produto_formatada,
    ROW_NUMBER() OVER (
      PARTITION BY LOWER(REPLACE(ds_produto, ' ', '')), id_subcategoria
    ) AS qtde
  FROM `luizalabs-data-challenge.staging_corrigido.produto`
)

SELECT *
FROM cte_produtos_duplicados cd
WHERE cd.qtde > 1

## Tabela Item Pedido

In [ ]:
%%bigquery
WITH cte_item_pedido_duplicados AS (
  SELECT
      id_produto,
      id_pedido,
      quantidade,
      vr_unitario,
    ROW_NUMBER() OVER (
      PARTITION BY
      id_produto,
      id_pedido,
      quantidade
    ) AS qtde
  FROM `luizalabs-data-challenge.staging_corrigido.item_pedido`
)

SELECT *
FROM cte_item_pedido_duplicados cd
WHERE cd.qtde > 1
